In [1]:
from numpy.random import seed
seed(1337)
from tensorflow import set_random_seed
set_random_seed(2674)
import numpy as np
import h5py
import matplotlib
matplotlib.use('Qt4Agg')
import matplotlib.pyplot as plt
from scipy import integrate, interpolate, signal, optimize, stats
import cPickle as pickle
import lal
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, BatchNormalization, Flatten
from keras.optimizers import Nadam, SGD
from keras.callbacks import ModelCheckpoint
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pyfftw
import progressbar
import time
from sklearn import metrics
import itertools
np.set_printoptions(edgeitems=30, linewidth=160)
import warnings
warnings.filterwarnings('ignore')
import pickle

Using TensorFlow backend.


In [ ]:
# This code is for reading simulated SNe waveforms
# This code will apply shift to the waveform 
# samples so that the waveform will always be in the certre +- user customized percentage.

In [2]:
# The name of the file that contains the simulated CCSN waveforms
filename = './Data/GWdatabase.h5'

# Read the simulated CCSN waveforms
waveformfile = h5py.File(filename, 'r')


# The first level keys of the h5 file
reduced_data = waveformfile.keys()[0]
waveformfilekey = waveformfile.keys()[1]
yeofrho = waveformfile.keys()[2]

waveformfamily = []
waveformfamily_keys = []

# Since there are 1824 different simulated CCSN waveform. 
# Each of which is saved in a different waveformfile key 
# So the loop below is to retreive all the keys with which the waveform strain data is accessed,
# and save it to waveformfamily.
# Each waveform family has 5 different keys, so the second part is to retrieve these 5 keys, and save them
# to waveformfamily_keys.

for i, key in enumerate(waveformfile[waveformfilekey].keys()):
    waveformfamily.append(key)
    if i == 0:
        for j, _ in enumerate(waveformfile[waveformfilekey][waveformfamily[i]].keys()):
            waveformfamily_keys.append(waveformfile[waveformfilekey][waveformfamily[i]].keys()[j])
originalSNR = np.array(waveformfile[reduced_data][u'SNR(aLIGOfrom10kpc)'])

In [3]:
# This is to set some parameters for the training.
# Since the waveforms are stored in the unit of strain * distan
# the waveform amplitudes need to be divided by a distance.

# Convection factor between par sec and meters
PctMe = lal.PC_SI

# The distance the waveform will be divided by, in centimeters
Dist = 10.0 * 1e3 * PctMe * 1e2

# Since the waveform samples come in different lengths, 
# so every waveform sample will be set to the longest length.
# findmax/findmin is a variable that saves the longest/shortest length of the waveform samples.
# k/kmin is the index referring to the longest/shortest waveform sample.
findmax = 0
k = 0 
findmin = 1e10
kmin = 0
#length = np.zeros(len(waveformfamily))
#waveformfamily = [waveformfamily[0]]


# Since the waveform contains 1824 waveforms, which are different both in the morophology and the duration,
# training a network with all these waveforms may make it hard to debug. So one may want to limit the variation
# in the waveform samples by limiting the number of waveform samples put in the training. 
no_waves_considered = 1824#1824
for i in range(len(waveformfamily[0:no_waves_considered])):
    waveformnumber = i

    ts = np.array(waveformfile[waveformfilekey][waveformfamily[waveformnumber]][u't-tb(s)']) 
    #waves = np.array(waveforms[waveformkey][waveformfamily[waveformnumber]][u'strain*dist(cm)']) / Dist 
    if findmax < len(ts):
        findmax = len(ts)
        k = i
    if findmin > len(ts):
        findmin = len(ts)
        kmin = i

print(findmax, k, findmin, kmin)


(108507, 197, 13156, 1416)


In [4]:
# The simulated waveforms are sampled with a sampling rate equal to 65535 Hz, 
# coupled with the longest waveform is ~1.66s, this makes the longest waveform contains 1e5 elements. 
# Since this code will make other waveforms the same length as the longest length, this requires huge amount of memory,
# and makes training very slow and difficult. 
# Therefore, this codes uses scipy.signal.decimate to down sample the waveforms


def padandextractwave(waveformfile, waveformfilekey, waveformfamily, strainkey, wavemaxlength, Dist, no_waves_considered, R):
    # Number of simulated waveforms considered
    noofwaves = len(waveformfamily[0:no_waves_considered])
    
    msg = 'Reading waveforms from file and downsampling them by a factor of %s............' %(R)
    print(msg)
    bar = progressbar.ProgressBar(max_value = no_waves_considered)
    
    # downsample factor, the downsampled waveform will have length = original length / R
    
    # Vector used to save the downsampled waveform
    downsampled_waveforms = np.array([np.zeros(wavemaxlength / R) for i in range(noofwaves)])
    
    for i, whichsimulation in enumerate(waveformfamily[0:no_waves_considered]):
        
        # convert the unit of the waveform from strain*distance to strain
        wave = np.array(waveformfile[waveformfilekey][whichsimulation][strainkey]) / Dist
        wavelength = len(wave)
        
        # Pad the waveform with zero so that it has the same length as the longest waveform, 
        # or whatever length is set by wavemaxlength
        temporary = np.pad(wave, (0, wavemaxlength - wavelength), 'constant', constant_values = 0)
        
        # down sample
        downsampled_waveforms[i] = signal.decimate(temporary, R, ftype='iir')
        bar.update(i + 1)
        
    return downsampled_waveforms
    

In [5]:

# Since the original longest waveform length may not be dividable by the down sample vector, 
# this is to ensure that the length will be dividable. 
R = 16
findmax = 108512

findmax = np.ceil(findmax/8.0) * 8

# the assumed observation/simulation duration for every waveform 
Tobs = findmax / 65535.0
#start = time.time()
SNewaves = padandextractwave(waveformfile, waveformfilekey, waveformfamily, u'strain*dist(cm)', int(findmax), Dist, no_waves_considered, R)
#elapsed = time.time() - start
#print(elapsed)
# Using the downsampled waveform to compute the new sampling rate
New_sr = (len(SNewaves[0]) - 1) / Tobs
# the new spacing in time
New_dt = 1.0 / New_sr



N/A% (0 of 1824) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

Reading waveforms from file and downsampling them by a factor of 16............


 99% (1823 of 1824) |################### | Elapsed Time: 0:01:02 ETA:   0:00:00

In [6]:
def ASDtxt(x):
    """This function reads the following noise curves given a detector name."""
    return {
        'LET':'./ASD/ET_D.txt',
        'LCE':'./ASD/CE.txt',
        'H1': './ASD/ligoII_NS.txt',
        'L1': './ASD/ligoII_NS.txt',
        'V1': './ASD/virgoII.txt',
        'I2': './ASD/ligoII_NS.txt',
        'KAGRA': './ASD/ligoII_NS.txt',
        'ET_1': './ASD/ET_D.txt',
        'ET_2': './ASD/ET_D.txt',
        'ET_3': './ASD/ET_D.txt',
        'A2': './ASD/ligoII_NS.txt',
        'A2.5': './ASD/ligoII_NS.txt',
    }[x]


In [7]:
def readnos(detector, f_points):
    """This function interpolates the noise given the frequency samples."""
    nos_file = ASDtxt(detector)
    f_str = []
    ASD_str = []
    file = open(nos_file, 'r')
    readFile = file.readlines()
    file.close()
    f = []
    ASD = []
    
    for line in readFile:
        p = line.split()
        f_str.append(float(p[0]))
        ASD_str.append(float(p[1]))
    f = np.log10(np.array(f_str))
    ASD = np.log10(np.array(ASD_str))
    nosinterpolate = interpolate.splrep(f, ASD, w=1.0*np.ones(len(ASD)), s=0)
    
    nos = interpolate.splev(np.log10(f_points), nosinterpolate, der = 0, ext = 3)
    nos = 10**nos
    
    return nos

In [8]:
def noisegenerator(Tobs, det, SR, df, dt):
    """This function generates noise based on amplitude spectral density"""
    
    # The number of time stamps
    Ns = Tobs * SR 
    
    # The number of the frequency samples
    Nf = int(Ns // 2 + 1)
    
    # The frequency sample
    fs = np.arange(Nf) * df
    
    # read ASD
    ASD = readnos(det, fs)
    #plt.loglog(fs, ASD)
    #plt.show()
    #dd
    
    PSD = ASD ** 2
    # scale the ASD by the observation time, and this will be the highest amplitude of the generated noise
    Amp = np.sqrt(0.25 * Tobs * PSD)
    
    
    idx = np.argwhere(PSD==0.0)
    Amp[idx] = 0.0
    
    real_nos = Amp * np.random.normal(0.0, 1.0, Nf)
    img_nos = Amp * np.random.normal(0.0, 1.0, Nf)
    
    # This is to ensure there is no strange behaviour from noise at low frequency.
    # This is because the interpolation function will interpolate strange values at frequencies betweem 1 - 10Hz.
    #low_cutoff = 20
    #high_cutoff = 2048
    
    #idx_1 =  int(low_cutoff/df)
    #real_nos[0:idx_1] = 0
    #img_nos[0:idx_1] = 0
    #idx_2 = int(high_cutoff/df)
    #real_nos[idx_2:] = 0
    #img_nos[idx_2:] = 0
    
    nos = real_nos + 1j * img_nos

    
    # Fourier transiform converts the generated noise to the tme domain
    fftinput = pyfftw.empty_aligned(len(nos), dtype='complex128')
    
    fft_object = pyfftw.builders.irfft(fftinput)

    nos_realization = Ns* fft_object(nos) * df

    return ASD, nos_realization, fs
    

In [9]:
def SNR_calculator(waves_in_time_domain, dt, Det):
    
    length = len(waves_in_time_domain)
    
    df = 1.0 / (length * dt)
    
    Nf = int((length // 2 + 1))
    
    fftinput_for_snr = pyfftw.empty_aligned(length, dtype='complex128')     
    fft_object_for_snr = pyfftw.builders.rfft(fftinput_for_snr)      
     
    # frequency samples
    fs = np.arange(Nf) * df
    
    # Amplitude spectral density
    ASD = readnos(Det, fs)
        
    temporary_wave_in_f = fft_object_for_snr(waves_in_time_domain) * dt
    
    snr = np.sqrt( 4.0 * sum( abs(temporary_wave_in_f) ** 2 / ASD ** 2 ) * df )
    
    return snr
    

In [10]:
def rescale_to_set_SNR(preset_SNR, SNewaves, dt, Det):
    
    df = 1.0 / (len(SNewaves[0]) * dt)
    fftinput_for_snr = pyfftw.empty_aligned(len(SNewaves[0]), dtype='complex128')     
    fft_object_for_snr = pyfftw.builders.rfft(fftinput_for_snr)      
    
    Nf = int((len(SNewaves[0]) // 2 + 1))
    
    # frequency samples
    fs = np.arange(Nf) * df
    
    # Amplitude spectral density
    ASD = readnos(Det, fs)
    msg = 'Rescaling the amplitude of the waveforms so that their optimal SNR is %s.........' %(preset_SNR)
    print(msg)
    print(" ")
    bar = progressbar.ProgressBar(max_value = len(SNewaves))
    
    for i, wave in enumerate(SNewaves):
        temporary_wave_in_f = fft_object_for_snr(wave) * dt
        temporary_snr = np.sqrt( 4.0 * sum( abs(temporary_wave_in_f) ** 2 / ASD ** 2 ) * df )
        SNR_factor = preset_SNR / temporary_snr
    
        SNewaves[i] = SNR_factor * wave
        #print(temporary_snr)
        #print(  np.sqrt(4.0 * sum(abs(fft_object_for_snr(SNewaves[i]) * dt) **2 / ASD ** 2) * df))
        bar.update(i)
    
    return SNewaves
    

In [11]:
def data_generator(seed, ts, dt, Sr, percentage, Det, SNewaves, N_rz, multiplication):
    """This function generates the data for training/validation/testing."""
    
    np.random.seed(seed)
    
    # The number of sample will be equal to the number of N_rz(noise realizations)
    data = np.array([np.zeros_like(ts) for i in range(N_rz)])
    
    # Signal to noise ratio
    #SNR = np.zeros(N_rz)
    
    # Number of time stamps
    Ns = len(ts)
    
    # Number of frequency samples
    Nf = int(Ns //2 + 1)
    
    # Observation time
    Tobs = ts[-1] + dt
    
    # spacing in the frequency domain
    df = 1.0/Tobs
    # frequency samples
    fs = np.arange(Nf) * df
    
    # Amplitude spectral density
    ASD = readnos(Det, fs)
    
    
    toolbar_width = N_rz

    
    
    msg = 'Generating noise realizations.......'
    print(msg)
    print(" ")
    # setup toolbar
    bar = progressbar.ProgressBar(max_value=toolbar_width)
    

    
    # Generate noise
    for i in range(N_rz):
        #if (i+1) % 1000 == 0 & i != N_rz - 1:
        #   msg = 'The %s th to %s th noise realizations are now being generated.' %(i+1, i+1000)
        #    print(msg)
        _, data[i], _ = noisegenerator(Tobs, Det, Sr, df, dt)
        bar.update(i+1)



    msg = 'Adding noise to signals and converting them back to the time domain after whitening them in the frequency domain.....'
    print(msg)
    print(" ")
    bar_2 = progressbar.ProgressBar(max_value=toolbar_width)
    
    
    if ts[-1] == signal_duration:   

        for i in range(multiplication):
            for j in range(len(SNewaves)):

                count = i * len(SNewaves) + j
                #if (count + 1) % 1000 == 0 and count < 4999:
                #    msg = 'The %s th to %s th samples of the data set are now being generated.' %(count + 1,count + 1000)
                #    print(msg)
                data[count] += SNewaves[j]


                fftinput_1 = pyfftw.empty_aligned(len(data[count]), dtype='complex128')
                fft_object_1 = pyfftw.builders.rfft(fftinput_1)
                temporary = fft_object_1(data[count]) * 1.0/Sr
                temporary = temporary / ASD 


                #SNR[count] = np.sqrt(4.0 * sum(abs(temporary[int(100/df): int(500/df)]) ** 2 * df))
                #SNR_factor = SNR_set / SNR[count]
                #temporary = temporary * SNR_factor
                #if SNR_factor > 1:
                #    print(SNR_factor,count) 
                #print(SNR_factor, np.sqrt(4.0 * sum(abs(temporary) ** 2 * df)))
                fftinput_2 = pyfftw.empty_aligned(len(temporary), dtype='complex128')
                fft_object_2 = pyfftw.builders.irfft(fftinput_2)
                data[count] = Ns * fft_object_2(temporary) * df * np.sqrt(2.0/ Sr)
                bar_2.update( count + 1)
    elif ts[-1] > signal_duration:
        for i in range(multiplication):
            for j in range(len(SNewaves)):

                count = i * len(SNewaves) + j
                #if (count + 1) % 1000 == 0 and count < 4999:
                #    msg = 'The %s th to %s th samples of the data set are now being generated.' %(count + 1,count + 1000)
                #    print(msg)
                # This is to draw a random and determine     
                random_shift_percentage = np.random.uniform(-percentage, percentage)
                original_starting_point = sample_length / 2 - signal_length / 2
                shifted_starting_point = int(original_starting_point * (1 + random_shift_percentage))
                
                data[count][shifted_starting_point: shifted_starting_point + signal_length] = data[count][shifted_starting_point: shifted_starting_point + signal_length] + SNewaves[j]
        
                fftinput_1 = pyfftw.empty_aligned(len(data[count]), dtype='complex128')
                fft_object_1 = pyfftw.builders.rfft(fftinput_1)
                temporary = fft_object_1(data[count]) * 1.0 / Sr
                temporary = temporary / ASD 


                #SNR[count] = np.sqrt(4.0 * sum(abs(temporary[int(100/df): int(500/df)]) ** 2 * df))
                #SNR_factor = SNR_set / SNR[count]
                #temporary[int(100/df): int(500/df)] = temporary[int(100/df): int(500/df)] * SNR_factor
                
                #if SNR_factor > 1:
                #    print(SNR_factor,count) 
                #print(SNR_factor, np.sqrt(4.0 * sum(abs(temporary) ** 2 * df)))
                
                fftinput_2 = pyfftw.empty_aligned(len(temporary), dtype='complex128')
                fft_object_2 = pyfftw.builders.irfft(fftinput_2)
                data[count] = Ns * fft_object_2(temporary) * df * np.sqrt(2.0/ Sr)
                bar_2.update( count + 1 )
    else:
        raise Exception('The sample length should be longer than or equal to the signal length') 

            
    for i in range(multiplication * len(SNewaves), N_rz):
        fftinput_1 = pyfftw.empty_aligned(len(data[i]), dtype='complex128')
        fft_object_1 = pyfftw.builders.rfft(fftinput_1)
        temporary = fft_object_1(data[i]) *  1.0 / Sr 
        temporary = temporary / ASD 
        
        fftinput_2 = pyfftw.empty_aligned(len(temporary), dtype='complex128')
        fft_object_2 = pyfftw.builders.irfft(fftinput_2)
        data[i] = Ns * fft_object_2(temporary) * df * np.sqrt(2.0/ Sr)
        bar_2.update(i + 1)
            
            
    return data #SNR
        


In [12]:
def whiten_data(not_whitened_data_in_time_domain, ASD, dt, SR):
    
    num = len(not_whitened_data_in_time_domain)
    signal_len = len(not_whitened_data_in_time_domain[0])
    
    whitened_data = np.array([np.zeros(signal_len ) for i in range(num)])
    fftinput_in_td = pyfftw.empty_aligned(signal_len, dtype='complex128')
    fft_object_to_f = pyfftw.builders.rfft(fftinput_in_td)
    
    
    fftinput_in_fd = pyfftw.empty_aligned(signal_len//2 + 1, dtype='complex128')
    fft_object_to_t = pyfftw.builders.irfft(fftinput_in_fd)
    
    for i, nwd in enumerate(not_whitened_data_in_time_domain):
        temp = fft_object_to_f(nwd) * dt / ASD
        whitened_data[i] = fft_object_to_t(temp) * np.sqrt(2.0/ SR) / dt
        
    
    return whitened_data 



In [ ]:
asd, _, _ = noisegenerator(ts[-1] + New_dt , 'H1', New_sr, 1.0/signal_duration, New_dt)
whitened_data = whiten_data(SNewaves, asd, New_dt, New_sr)


In [13]:
# the time stamps 
signal_length = len(SNewaves[0])
signal_duration = (signal_length - 1) * New_dt

# applying pad to make the sample longer. This is for the purpose of shifting the signal, so that the signal will appear to be in the centre +- user customised percentage
# If no padding is to be applied
sample_length = signal_length * 1.0

# time stamps after pad
ts = np.arange(sample_length) * New_dt
sample_duration = ts[-1]

In [24]:
SNR_set = 6.0
SNewaves = rescale_to_set_SNR(SNR_set, SNewaves, New_dt, 'H1')
plt.plot(SNewaves[0])
plt.show()

  6% (117 of 1824) |#                    | Elapsed Time: 0:00:00 ETA:   0:00:01

Rescaling the amplitude of the waveforms so that their optimal SNR is 6.0.........
 


 98% (1803 of 1824) |################### | Elapsed Time: 0:00:01 ETA:   0:00:00

In [25]:
if 'data' in globals():
    del data
    del label
if 'train_sample' in globals():
    del train_sample, train_label, val_sample, val_label, test_sample, test_label
    
multiplication = 6
presence = len(SNewaves) * multiplication
shift_percentage = 0.0
seed = 10
Det = 'H1'

# Number of noise realization. This will be the final number of data samples for training + validation + testing
N_rz = 20000
data = data_generator(seed, ts, New_dt, New_sr, shift_percentage, Det, SNewaves, N_rz, multiplication)
label = np.concatenate((np.ones(presence), np.zeros(N_rz - presence)))

  0% (24 of 20000) |                     | Elapsed Time: 0:00:00 ETA:   0:01:26

Generating noise realizations.......
 


  0% (91 of 20000) |                     | Elapsed Time: 0:00:00 ETA:   0:00:22

Adding noise to signals and converting them back to the time domain after whitening them in the frequency domain.....
 


 99% (19961 of 20000) |################# | Elapsed Time: 0:00:22 ETA:   0:00:00

In [21]:
plt.plot(SNewaves[339])
plt.show()

In [159]:
pathandname='showdata.pkl'
fp = open(pathandname,"w")
pickle.dump([ts, show_data], fp)
fp.close()

In [26]:
def shuffle_data(sample, label,  shuffle_times, seed):
    np.random.seed(seed)
    for i in range(shuffle_times):
        state = np.random.randint(0,100)
        sample, label = shuffle(sample, label, random_state=state)
        
    return sample, label


In [27]:
def split_data(sample, label, savefortest, presence, shuffled, tv_split):
    
    data_size = len(sample) 
    
    
    if shuffled == True:
        split_point = int((data_size - 2 * savefortest) * tv_split)
        train_sample = sample[0 : split_point] 
        train_label = label[0 : split_point]
        
        val_sample = sample[split_point : data_size - 2 * savefortest]
        val_label = label[split_point : data_size - 2 * savefortest]
    
        test_sample = sample[data_size - 2 * savefortest : ]
        test_label = label[data_size - 2 * savefortest : ]

    else:

        nos_portion = (data_size - presence - savefortest)/2
        nos_start = presence

        
        train_start_index_signal = 0
        train_end_index_signal = (presence - savefortest) / 2

        train_start_index_noise = nos_start
        train_end_index_noise = nos_start + nos_portion


        val_start_index_signal = train_end_index_signal
        val_end_index_signal = presence - savefortest

        val_start_index_noise = train_end_index_noise
        val_end_index_noise = train_end_index_noise + nos_portion


        test_start_index_signal = val_end_index_signal
        test_end_index_signal =  val_end_index_signal + savefortest

        test_start_index_noise = val_end_index_noise
        test_end_index_noise =  val_end_index_noise + savefortest

    
        train_sample = np.concatenate((data[train_start_index_signal:train_end_index_signal], 
                                       data[train_start_index_noise:train_end_index_noise]))

        train_label = np.concatenate((np.ones(train_end_index_signal - train_start_index_signal), 
                                      np.zeros(train_end_index_noise - train_start_index_noise)))

        val_sample = np.concatenate((data[val_start_index_signal:val_end_index_signal], 
                                    data[val_start_index_noise:val_end_index_noise]))

        val_label = np.concatenate((np.ones(val_end_index_signal - val_start_index_signal), 
                                    np.zeros(val_end_index_noise - val_start_index_noise)))

        test_sample = np.concatenate((data[test_start_index_signal : test_end_index_signal], 
                                     data[test_start_index_noise : test_end_index_noise]))

        test_label = np.concatenate((np.ones(test_end_index_signal - test_start_index_signal), 
                                    np.zeros(test_end_index_noise - test_start_index_noise)))
        
        #print(train_start_index_signal,train_end_index_signal, train_start_index_noise , train_end_index_noise, 
        #      val_start_index_signal, val_end_index_signal, val_start_index_noise, val_end_index_noise,
        #      test_start_index_signal, test_end_index_signal, test_start_index_noise, test_end_index_noise)
    return train_sample, train_label, val_sample, val_label, test_sample, test_label

In [28]:
presence = len(SNewaves) * multiplication
label = np.concatenate((np.ones(presence), np.zeros(N_rz - presence)))

shuffle_times = 1
seed_s = 387
sample, label = shuffle_data(data, label,  shuffle_times, seed_s)
savefortest = 100
shuffled = True
tv_split = 0.8
train_sample, train_label, val_sample, val_label, test_sample, test_label = split_data(sample, label, savefortest, presence, shuffled, tv_split)
del sample

In [29]:
"""Below is the CNN part of this code""" 

batch_size = 30      # number of time series per batch
num_classes = 2      # signal or background
epochs = 20          # number of full passes of the dataset
outdir = './results' # directory to store results in



In [30]:
number_of_sample_for_training = len(train_sample)
number_of_sample_for_testing = len(test_sample)
number_of_sample_for_validation = len(val_sample)
training_sample_length = len(train_sample[0])

train_sample = train_sample.reshape(number_of_sample_for_training, 1, training_sample_length)
test_sample = test_sample.reshape(number_of_sample_for_testing, 1, training_sample_length)
val_sample = val_sample.reshape(number_of_sample_for_validation, 1, training_sample_length)

In [31]:
keras.backend.set_image_data_format('channels_first')
training_sample_length = len(train_sample[0][0])

train_sample = train_sample.reshape(-1, 1, 1, training_sample_length)
val_sample = val_sample.reshape(-1, 1, 1, training_sample_length)
test_sample = test_sample.reshape(-1, 1, 1, training_sample_length)

input_shape = train_sample.shape[1:]

In [32]:
train_label = keras.utils.to_categorical(train_label , num_classes)
val_label = keras.utils.to_categorical(val_label, num_classes)
test_label = keras.utils.to_categorical(test_label, num_classes)

In [52]:
model = Sequential()    # define the type of keras model

# add the layers
# conv1
model.add(Conv2D(8, (1,64), activation='elu', input_shape=input_shape))
# maxpool2
model.add(MaxPool2D((1,8)))
# conv2

model.add(Conv2D(16, (1,16), activation='elu'))
# maxpool2
model.add(MaxPool2D((1,6)))
# the input the fully connected layer must be 1-D vector
model.add(Flatten())
model.add(Dense(32, activation='elu'))
#model.add(Dropout(0.5))
dol = keras.layers.Dropout(0.5, noise_shape=None, seed=10)

model.add(dol)
# add the output layer with softmax actiavtion for classication
model.add(Dense(num_classes, activation='softmax'))
# print a summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_103 (Conv2D)          (None, 8, 1, 6719)        520       
_________________________________________________________________
max_pooling2d_103 (MaxPoolin (None, 8, 1, 839)         0         
_________________________________________________________________
conv2d_104 (Conv2D)          (None, 16, 1, 824)        2064      
_________________________________________________________________
conv2d_105 (Conv2D)          (None, 16, 1, 809)        4112      
_________________________________________________________________
max_pooling2d_104 (MaxPoolin (None, 16, 1, 134)        0         
_________________________________________________________________
flatten_52 (Flatten)         (None, 2144)              0         
_________________________________________________________________
dense_103 (Dense)            (None, 32)                68640     
__________

In [60]:
del model


In [61]:
model = Sequential()    # define the type of keras model

# add the layers
# conv1
model.add(Conv2D(8, (1,64), activation='elu', input_shape=input_shape))
# maxpool2
model.add(MaxPool2D((1,8)))
# conv2
model.add(Conv2D(16, (1,16), activation='elu'))
# maxpool2
model.add(MaxPool2D((1,6)))
# the input the fully connected layer must be 1-D vector
model.add(Flatten())
model.add(Dense(32, activation='elu'))
#model.add(Dropout(0.5))
dol = keras.layers.Dropout(0.5, noise_shape=None, seed=10)

model.add(dol)
# add the output layer with softmax actiavtion for classication
model.add(Dense(num_classes, activation='softmax'))
# print a summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 8, 1, 6719)        520       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 8, 1, 839)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 16, 1, 824)        2064      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 16, 1, 137)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 2192)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 32)                70176     
_________________________________________________________________
dropout_9 (Dropout)          (None, 32)                0         
__________

# Test 1. 

model.add(Conv2D(8, (1,64), activation='elu', input_shape=input_shape))
# maxpool2
model.add(MaxPool2D((1,8)))


model.add(Conv2D(16, (1,16), activation='elu'))
# maxpool2
model.add(MaxPool2D((1,6)))


# the input the fully connected layer must be 1-D vector
model.add(Flatten())
model.add(Dense(32, activation='elu'))

dol = keras.layers.Dropout(0.5, noise_shape=None, seed=10)

model.add(dol)
# add the output layer with softmax actiavtion for classication
model.add(Dense(num_classes, activation='softmax'))


# Test 2. 

model.add(Conv2D(8, (1,128), activation='elu', input_shape=input_shape))
# maxpool2
model.add(MaxPool2D((1,8)))


model.add(Conv2D(16, (1,16), activation='elu'))
# maxpool2
model.add(MaxPool2D((1,6)))


# the input the fully connected layer must be 1-D vector
model.add(Flatten())
model.add(Dense(32, activation='elu'))

dol = keras.layers.Dropout(0.5, noise_shape=None, seed=10)

model.add(dol)
# add the output layer with softmax actiavtion for classication
model.add(Dense(num_classes, activation='softmax'))



# Test 3. 

model.add(Conv2D(16, (1,64), activation='elu', input_shape=input_shape))
# maxpool2
model.add(MaxPool2D((1,8)))


model.add(Conv2D(16, (1,16), activation='elu'))
# maxpool2
model.add(MaxPool2D((1,6)))


# the input the fully connected layer must be 1-D vector
model.add(Flatten())
model.add(Dense(32, activation='elu'))

dol = keras.layers.Dropout(0.5, noise_shape=None, seed=10)

model.add(dol)
# add the output layer with softmax actiavtion for classication
model.add(Dense(num_classes, activation='softmax'))

In [33]:
def displaymsg(x):
    rmd = x % 10
    if rmd <= 3 and rmd !=0 and x != 11 and x != 12 and x!= 13:
        
        return {
            1: 'st',
            2: 'nd',
            3: 'rd',
        }[rmd]
    else:
        return 'th'


In [95]:
(len(SNewaves[0]) - 127) / 8.0

831.875

In [53]:
train_times = 10
loss = np.array([np.zeros(epochs) for i in range(train_times)])
acc = np.array([np.zeros(epochs) for i in range(train_times)])
val_loss= np.array([np.zeros(epochs) for i in range(train_times)])
val_acc = np.array([np.zeros(epochs) for i in range(train_times)])
test_loss = np.zeros(train_times)
test_acc = np.zeros(train_times)

for i in range(train_times):
    msg = "Training for the %s%s time." %(i+1, displaymsg(i+1))
    print(msg)    
    if "model" in globals():
        del model
        
    model = Sequential()    # define the type of keras model

    # add the layers
    # conv1
    model.add(Conv2D(8, (1,64), activation='elu', input_shape=input_shape))
    # maxpool2
    model.add(MaxPool2D((1,64)))
 
    model.add(Conv2D(8, (1,16), activation='elu'))
    model.add(Conv2D(8, (1,4), activation='elu'))
    # maxpool2
    model.add(MaxPool2D((1,6)))


    # the input the fully connected layer must be 1-D vector
    model.add(Flatten())
    model.add(Dense(32, activation='elu'))

    dol = keras.layers.Dropout(0.5, noise_shape=None, seed=10)

    model.add(dol)
    # add the output layer with softmax actiavtion for classication
    model.add(Dense(num_classes, activation='softmax'))
    # print a summary
    #model.summary()
    model.compile(loss='categorical_crossentropy', optimizer= Nadam(), metrics=['accuracy'])
    weight_file_name = './results/best_weights_at_SNR_%s.hdf5' %(int(SNR_set))
    modelCheck = ModelCheckpoint(weight_file_name.format(outdir), monitor='val_acc', verbose=0, save_best_only=True,save_weights_only=True, mode='auto', period=0)
    
    history = model.fit(train_sample, train_label, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(val_sample, val_label), callbacks = [modelCheck], shuffle = False)
    weight_file_name = '{0}/best_weights_at_SNR_%s.hdf5' %(int(SNR_set)) 
    model.load_weights(weight_file_name.format(outdir))
    # evaluate
    eval_results = model.evaluate(test_sample, test_label, sample_weight=None, batch_size=batch_size, verbose=1)
    print('Test loss:', eval_results[0])
    print('Test accuracy:', eval_results[1])
    test_loss[i] = eval_results[0]
    test_acc[i] = eval_results[1]
    
    loss[i] = np.array(history.history['loss'])
    acc[i] = np.array(history.history['acc'])
    
    val_loss[i] = np.array(history.history['val_loss'])
    val_acc[i] = np.array(history.history['val_acc'])

Training for the 1st time.
Train on 15840 samples, validate on 3960 samples
Epoch 1/20
15840/15840 [==============================] - 10s 640us/step - loss: 0.6286 - acc: 0.6165 - val_loss: 0.4383 - val_acc: 0.7914
Epoch 2/20
15840/15840 [==============================] - 6s 354us/step - loss: 0.4155 - acc: 0.8086 - val_loss: 0.3630 - val_acc: 0.8364
Epoch 3/20
15840/15840 [==============================] - 5s 343us/step - loss: 0.3673 - acc: 0.8402 - val_loss: 0.3489 - val_acc: 0.8475
Epoch 4/20
15840/15840 [==============================] - 6s 350us/step - loss: 0.3406 - acc: 0.8556 - val_loss: 0.3400 - val_acc: 0.8508
Epoch 5/20
15840/15840 [==============================] - 6s 348us/step - loss: 0.3266 - acc: 0.8624 - val_loss: 0.3545 - val_acc: 0.8556
Epoch 6/20
15840/15840 [==============================] - 5s 347us/step - loss: 0.3142 - acc: 0.8710 - val_loss: 0.3602 - val_acc: 0.8505
Epoch 7/20
15840/15840 [==============================] - 5s 343us/step - loss: 0.3051 - acc: 0

15840/15840 [==============================] - 5s 334us/step - loss: 0.2658 - acc: 0.8910 - val_loss: 0.3617 - val_acc: 0.8495
Epoch 17/20
15840/15840 [==============================] - 5s 327us/step - loss: 0.2609 - acc: 0.8939 - val_loss: 0.3509 - val_acc: 0.8535
Epoch 18/20
15840/15840 [==============================] - 5s 337us/step - loss: 0.2579 - acc: 0.8944 - val_loss: 0.3560 - val_acc: 0.8503
Epoch 19/20
15840/15840 [==============================] - 5s 337us/step - loss: 0.2564 - acc: 0.8961 - val_loss: 0.3551 - val_acc: 0.8487
Epoch 20/20
200/200 [==============================] - 0s 141us/step
('Test loss:', 0.29612554907798766)
('Test accuracy:', 0.8799999862909317)
Training for the 4th time.
Train on 15840 samples, validate on 3960 samples
Epoch 1/20
15840/15840 [==============================] - 10s 656us/step - loss: 0.6268 - acc: 0.6222 - val_loss: 0.4661 - val_acc: 0.7657
Epoch 2/20
15840/15840 [==============================] - 6s 347us/step - loss: 0.4073 - acc: 0.8

Epoch 11/20
15840/15840 [==============================] - 5s 343us/step - loss: 0.2862 - acc: 0.8845 - val_loss: 0.3202 - val_acc: 0.8652
Epoch 12/20
15840/15840 [==============================] - 5s 337us/step - loss: 0.2837 - acc: 0.8851 - val_loss: 0.3249 - val_acc: 0.8641
Epoch 13/20
15840/15840 [==============================] - 5s 334us/step - loss: 0.2781 - acc: 0.8874 - val_loss: 0.3197 - val_acc: 0.8649
Epoch 14/20
15840/15840 [==============================] - 6s 349us/step - loss: 0.2745 - acc: 0.8870 - val_loss: 0.3203 - val_acc: 0.8667
Epoch 15/20
15840/15840 [==============================] - 5s 320us/step - loss: 0.2710 - acc: 0.8917 - val_loss: 0.3222 - val_acc: 0.8649
Epoch 16/20
15840/15840 [==============================] - 5s 344us/step - loss: 0.2704 - acc: 0.8934 - val_loss: 0.3280 - val_acc: 0.8667
Epoch 17/20
15840/15840 [==============================] - 5s 341us/step - loss: 0.2648 - acc: 0.8930 - val_loss: 0.3364 - val_acc: 0.8631
Epoch 18/20
15840/15840 [==

15840/15840 [==============================] - 6s 361us/step - loss: 0.3325 - acc: 0.8585 - val_loss: 0.3395 - val_acc: 0.8538
Epoch 6/20
15840/15840 [==============================] - 5s 330us/step - loss: 0.3209 - acc: 0.8631 - val_loss: 0.3459 - val_acc: 0.8515
Epoch 7/20
15840/15840 [==============================] - 6s 348us/step - loss: 0.3159 - acc: 0.8672 - val_loss: 0.3529 - val_acc: 0.8497
Epoch 8/20
15840/15840 [==============================] - 5s 342us/step - loss: 0.3065 - acc: 0.8737 - val_loss: 0.3736 - val_acc: 0.8412
Epoch 9/20
15840/15840 [==============================] - 5s 327us/step - loss: 0.3017 - acc: 0.8732 - val_loss: 0.3364 - val_acc: 0.8576
Epoch 10/20
15840/15840 [==============================] - 5s 340us/step - loss: 0.2969 - acc: 0.8758 - val_loss: 0.3412 - val_acc: 0.8540
Epoch 11/20
15840/15840 [==============================] - 5s 334us/step - loss: 0.2924 - acc: 0.8768 - val_loss: 0.3461 - val_acc: 0.8571
Epoch 12/20
15840/15840 [==================

In [47]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_81 (Conv2D)           (None, 8, 1, 6719)        520       
_________________________________________________________________
max_pooling2d_81 (MaxPooling (None, 8, 1, 104)         0         
_________________________________________________________________
conv2d_82 (Conv2D)           (None, 8, 1, 101)         264       
_________________________________________________________________
max_pooling2d_82 (MaxPooling (None, 8, 1, 25)          0         
_________________________________________________________________
flatten_41 (Flatten)         (None, 200)               0         
_________________________________________________________________
dense_81 (Dense)             (None, 32)                6432      
_________________________________________________________________
dropout_41 (Dropout)         (None, 32)                0         
__________

In [54]:
val_loss_model_1 = val_loss
val_acc_model_1 = val_acc
loss_model_1 = loss
acc_model_1 = acc
test_loss_model_1 = test_loss
test_acc_model_1 = test_acc

mean_test_loss_model_1 = np.mean(test_loss)
std_test_loss_model_1= np.std(test_loss)
mean_test_acc_model_1 = np.mean(test_acc)
std_test_acc_model_1= np.std(test_acc)


msg_1 = ''.join(["The mean of test_loss is : ", str(mean_test_loss_model_1)])
print(msg_1)

msg_1 = ''.join(["The std of test_loss is : ", str(std_test_loss_model_1)])
print(msg_1)



msg_1 = ''.join(["The mean of test_acc is : ", str(mean_test_acc_model_1)])
print(msg_1)

msg_1 = ''.join(["The std of test_acc is : ", str(std_test_acc_model_1)])
print(msg_1)

fig , axs = plt.subplots(2,1)

axs = axs.ravel()
# plot history
fontsize =20
counter = 0
for i in range(train_times):
    if counter == 0:
        axs[0].plot(np.arange(epochs)+1, loss_model_1[i], label = 'Loss', linewidth = 1, color = 'b')
        axs[0].plot(np.arange(epochs)+1, val_loss_model_1[i], label = 'Validation Loss', linewidth = 1, color = 'r')

        axs[1].plot(np.arange(epochs)+1, acc_model_1[i], label = 'Accuracy', linewidth = 1, color =  'b')
        axs[1].plot(np.arange(epochs)+1, val_acc_model_1[i], label = 'Validation Accurarcy', linewidth = 1, color =  'r')
        # set labels
        axs[0].set_ylabel('Loss', fontsize = fontsize)
        axs[1].set_xlabel('Epoch', fontsize = fontsize)
        axs[1].set_ylabel('Acc', fontsize = fontsize)



        # legends
        axs[0].legend(fontsize = fontsize)
        axs[1].legend(fontsize = fontsize)
    else:
        axs[0].plot(np.arange(epochs)+1, loss_model_1[i], linewidth = 1, color = 'b')
        axs[0].plot(np.arange(epochs)+1, val_loss_model_1[i], linewidth = 1, color = 'r')

        axs[1].plot(np.arange(epochs)+1, acc_model_1[i], linewidth = 1, color = 'b')
        axs[1].plot(np.arange(epochs)+1, val_acc_model_1[i], linewidth = 1, color = 'r')
        # set labels
        axs[0].set_ylabel('Loss', fontsize = fontsize)
        axs[1].set_xlabel('Epoch', fontsize = fontsize)
        axs[1].set_ylabel('Acc', fontsize = fontsize)



        # legends
        #axs[0].legend(fontsize = fontsize)
        #axs[1].legend(fontsize = fontsize)
    counter += 1
# grids
axs[0].grid()
axs[1].grid()
axs[0].set_xlim([1, 20])
axs[0].set_ylim([0, 1.14])#(bottom = 0)
axs[0].set_xticks(np.arange(epochs)+1)

axs[1].set_xlim([1, 20])
axs[1].set_ylim([0.54, 1])#(top = 1)
axs[1].set_xticks(np.arange(epochs)+1)

plt.subplots_adjust(left = 0.1, bottom = 0.1, right = 0.90, top = 0.95)
for ax in axs:
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        tick.label1.set_fontweight('normal')
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        tick.label1.set_fontweight('normal')
plt.show()

The mean of test_loss is : 0.2865222058817744
The std of test_loss is : 0.023830221216452733
The mean of test_acc is : 0.8794999912381172
The std of test_acc is : 0.01572418284901004


In [77]:
val_loss_model_2 = val_loss
val_acc_model_2 = val_acc
loss_model_2 = loss
acc_model_2 = acc
test_loss_model_2 = test_loss
test_acc_model_2 = test_acc

mean_test_loss_model_2 = np.mean(test_loss)
std_test_loss_model_2 = np.std(test_loss)
msg_2 = ''.join(["The mean of test_loss is : ", str(mean_test_loss_model_2)])
print(msg_2)

msg_2 = ''.join(["The std of test_loss is : ", str(std_test_loss_model_2)])
print(msg_2)


mean_test_acc_model_2 = np.mean(test_acc)
std_test_acc_model_2= np.std(test_acc)
msg_2 = ''.join(["The mean of test_acc is : ", str(mean_test_acc_model_2)])
print(msg_2)

msg_2 = ''.join(["The std of test_acc is : ", str(std_test_acc_model_2)])
print(msg_2)

fig , axs = plt.subplots(2,1)

axs = axs.ravel()
# plot history
fontsize =20
counter = 0
for i in range(train_times):
    if counter == 0:
        axs[0].plot(np.arange(epochs)+1, loss_model_2[i], label = 'Loss', linewidth = 1, color = 'b')
        axs[0].plot(np.arange(epochs)+1, val_loss_model_2[i], label = 'Validation Loss', linewidth = 1, color = 'r')

        axs[1].plot(np.arange(epochs)+1, acc_model_2[i], label = 'Accuracy', linewidth = 1, color =  'b')
        axs[1].plot(np.arange(epochs)+1, val_acc_model_2[i], label = 'Validation Accurarcy', linewidth = 1, color =  'r')
        # set labels
        axs[0].set_ylabel('Loss', fontsize = fontsize)
        axs[1].set_xlabel('Epoch', fontsize = fontsize)
        axs[1].set_ylabel('Acc', fontsize = fontsize)
        


        # legends
        axs[0].legend(fontsize = fontsize)
        axs[1].legend(fontsize = fontsize)
    else:
        axs[0].plot(np.arange(epochs)+1, loss_model_2[i], linewidth = 1, color = 'b')
        axs[0].plot(np.arange(epochs)+1, val_loss_model_2[i], linewidth = 1, color = 'r')

        axs[1].plot(np.arange(epochs)+1, acc_model_2[i], linewidth = 1, color = 'b')
        axs[1].plot(np.arange(epochs)+1, val_acc_model_2[i], linewidth = 1, color = 'r')
        # set labels
        axs[0].set_ylabel('Loss', fontsize = fontsize)
        axs[1].set_xlabel('Epoch', fontsize = fontsize)
        axs[1].set_ylabel('Acc', fontsize = fontsize)



        # legends
        #axs[0].legend(fontsize = fontsize)
        #axs[1].legend(fontsize = fontsize)
    counter += 1
# grids
axs[0].grid()
axs[1].grid()
axs[0].set_xlim([1, 20])
axs[0].set_ylim([0.5, 0.8])
axs[0].set_ylim(bottom = 0)
axs[0].set_xticks(np.arange(epochs)+1)
axs[1].set_xlim([1, 20])
axs[1].set_ylim(top = 1)
axs[1].set_xticks(np.arange(epochs)+1)

plt.subplots_adjust(left = 0.1, bottom = 0.1, right = 0.90, top = 0.95)
for ax in axs:
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        tick.label1.set_fontweight('normal')
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        tick.label1.set_fontweight('normal')
plt.show()

The mean of test_loss is : 0.6790371808409692
The std of test_loss is : 0.15195021316351015
The mean of test_acc is : 0.8004999956488609
The std of test_acc is : 0.018364366573549146


In [55]:
# load the best model
weight_file_name = '{0}/best_weights_at_SNR_%s.hdf5' %(int(6))

model.load_weights(weight_file_name.format(outdir))
# evaluate
eval_results = model.evaluate(test_sample, test_label,
                              sample_weight=None,
                              batch_size=batch_size, verbose=1)
print('Test loss:', eval_results[0])
print('Test accuracy:', eval_results[1])
signal_preds = model.predict(test_sample)


200/200 [==============================] - 0s 154us/step
('Test loss:', 0.299454515427351)
('Test accuracy:', 0.8599999904632568)


In [56]:
fa, ta, _ = metrics.roc_curve(test_label[:,1], signal_preds[:,1])
fig = plt.figure()
plt.plot(fa, ta, linewidth = 2, color = 'b')
plt.xlabel('False alarm probability',fontsize = fontsize)
plt.ylabel('True alarm probability',fontsize = fontsize)
plt.title('ROC curve for SNR %s'%(SNR_set), fontsize = fontsize)
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.subplots_adjust(left = 0.1, bottom = 0.1, right = 0.90, top = 0.95)

plt.grid()
ax = plt.gca()
for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(fontsize)
    tick.label1.set_fontweight('normal')
plt.show()

In [ ]:
fontsize = 20

fig , axs = plt.subplots(2,1, sharex = True)
axs = axs.ravel()
# plot history
axs[0].plot(history.history['loss'], label = 'Loss', linewidth = 2, color = 'b')
axs[0].plot(history.history['val_loss'], label = 'Validation Loss', linewidth = 2, color = 'r')
axs[1].plot(history.history['acc'], label = 'Accuracy', linewidth = 2, color = 'b')
axs[1].plot(history.history['val_acc'], label = 'Validation Accurarcy', linewidth = 2, color = 'r')
# set labels
axs[0].set_ylabel('Loss', fontsize = fontsize)
axs[1].set_xlabel('Epoch', fontsize = fontsize)
axs[1].set_ylabel('Acc', fontsize = fontsize)
# legends
axs[0].legend(fontsize = fontsize)
axs[1].legend(fontsize = fontsize)
# grids
axs[0].grid()
axs[1].grid()
axs[0].set_xlim([0, epochs])
axs[0].set_ylim(bottom = 0)

axs[1].set_xlim([0, epochs])
axs[1].set_ylim(top = 1)

plt.subplots_adjust(left = 0.1, bottom = 0.1, right = 0.90, top = 0.95)
for ax in axs:
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        tick.label1.set_fontweight('normal')
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        tick.label1.set_fontweight('normal')
plt.show()